In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
output.last_hidden_state.shape

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 检查是否有可用的 GPU，并设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用的设备: {device}")

# 1. 加载带有语言模型头的模型和分词器
# 将模型移动到指定的设备上
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

# 2. 准备输入提示
# 对于GPT-2这样的模型，提供故事的开头通常比给指令效果更好。
# 我们给它一个故事的引子，而不是一个命令。
prompt_text = "In a cozy little cottage, lived a fluffy cat named Whiskers. One sunny morning,"
prompt_text = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."

# 3. 对输入进行编码
# 将提示文本转换为模型可以理解的数字ID (tokens)，并将其发送到设备
input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(device)

# 4. 使用 model.generate() 生成文本
print("\n正在生成故事...")
# 调用 generate 方法来创作故事
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=200,          # 生成文本的最大长度（包含提示）
    num_return_sequences=1,  # 生成几个不同的故事
    no_repeat_ngram_size=2,  # 避免重复短语的关键参数
    do_sample=True,          # 启用采样，让文本更有创意，而不是死板的预测
    temperature=0.9,         # 控制创造性与确定性的平衡，数值越低越保守
    top_k=50,                # 采样时只考虑概率最高的50个词
    top_p=0.95,              # 核心采样，保留概率总和为95%的词汇
)

# 5. 解码生成的文本
# 将生成的数字ID序列转换回人类可读的字符串
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

# 6. 打印结果
print("\n--- 生成的故事 ---")
print(generated_text)
